![This is an image](Quant-Trading.jpg)

<font size="3">
Please visit our website <a href="https://www.quant-trading.co" target="_blank">quant-trading.co</a> for more tools on quantitative finance and data science.
</font>

# **How to make a treemap chart for market cap and daily returns?**

## **Treemap chart for market cap and daily returns**

<font size="3"> Many of you have seen treemap charts where the market cap defines the sizes of the boxes and daily returns the colors. In this notebook we will learn how to built a treemap chart for market cap and daily returns using python code. A lot of people think they need a fancy financial system like Bloomberg to get this kind of charts, but the reality is that you can make this treemap charts using public APIs like the one of <a href="https://quant-trading.co/how-to-download-data-from-yahoo-finance-api/" target="_blank">Yahoo Finance</a> and open code libraries like plotly. Let's see how to make a treemap for the stock market cap. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px

## **Get the information from the stocks you want to visualize**

<font size="3"> We will show a treemap of the S&P500 which contains 500 stocks. How can we get the tickers of all these stocks? An easy way to do it is looking at the information provided by ETFs that replicate than stock index. The ETF by BlackRock, IVV, replicates the S&P500 and has the information of all its constituents on their <a href="https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/" target="_blank">webpage</a>. We downloaded that data and store it into a csv file, which we read with the pandas library as you can see below:<br><br>

In [2]:
df = pd.read_csv('IVV.csv')

## **Get all the tickers**

<font size="3">We need to get all the tickers that are in the ETF. To do that we can simply use the function unique on the column Ticker as follows: <br><br>

In [3]:
ticker_list = df['Ticker'].unique()

## **Get the market cap, the day return and the sector for each ticker**

<font size="3">We are going to use the Yahoo Finance API to download that information. Remember, that if you want to learn how to use that API you can look in the following <a href="https://quant-trading.co/how-to-download-data-from-yahoo-finance-api/" target="_blank">link</a>. In this case we are looping through all the tickers in ticker_list, which is the python list we just defined<br><br>

In [4]:
df2 = pd.DataFrame(ticker_list,columns=['ticker'])
df2['mkt_cap'] = ''
df2['daily_return'] = ''
df2['sector'] = ''

i = 0

for ticker in ticker_list:
    my_ticker = yf.Ticker(ticker)   
    try:
        df2['mkt_cap'].iloc[i] = my_ticker.info['marketCap']        
    except:
        df2['mkt_cap'].iloc[i] = np.nan

    try:
        df2['daily_return'].iloc[i] = my_ticker.info['currentPrice'] / my_ticker.info['previousClose'] -1       
    except:
        df2['daily_return'].iloc[i] = np.nan
        
    try:
        df2['sector'].iloc[i] = my_ticker.info['sector']
    except:
        df2['sector'].iloc[i] = np.nan

    i = i + 1
    
df2.head(10)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XTSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XTSLA&crumb=urZuIFACEdg
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SGAFT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SGAFT&crumb=urZuIFACEdg
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ESU4?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ESU4&crumb=urZuIFACEdg


,ticker,mkt_cap,daily_return,sector
0,AAPL,3661198000128,-0.011388,Technology
1,MSFT,3140270161920,-0.012808,Technology
2,NVDA,3432028569600,-0.06217,Technology
3,AMZN,2335486771200,-0.024164,Consumer Cyclical
4,META,1559857201152,-0.019533,Communication Services
5,GOOG,2399744032768,-0.006314,Communication Services
6,BRKB,NaN,NaN,NaN
7,LLY,695432052736,0.010704,Healthcare
8,AVGO,1071718006784,-0.032867,Technology
9,TSLA,1265919197184,-0.040603,Consumer Cyclical


## **Delete empty roward and multiply the returns by 100**

<font size="3">we need to delete all the empty rows since that will give us some errors when making the treemap chart. We also want to multiply the returns by 100 to show that number in an easier way later. You can do that as follows<br><br>

In [5]:
df2 = df2[df2['mkt_cap'].notna()]
df2 = df2[df2['daily_return'].notna()]
df2 = df2[df2['sector'].notna()]

df2["daily_return"] = df2["daily_return"]*100
    
df2.head(10)

,ticker,mkt_cap,daily_return,sector
0,AAPL,3661198000128,-1.138776,Technology
1,MSFT,3140270161920,-1.280823,Technology
2,NVDA,3432028569600,-6.216958,Technology
3,AMZN,2335486771200,-2.416414,Consumer Cyclical
4,META,1559857201152,-1.953348,Communication Services
5,GOOG,2399744032768,-0.631441,Communication Services
7,LLY,695432052736,1.070448,Healthcare
8,AVGO,1071718006784,-3.286663,Technology
9,TSLA,1265919197184,-4.060333,Consumer Cyclical
10,JPM,684606226432,0.963255,Financial Services


## **Create a new colum with label all**

<font size="3">For a treemap chart it is very important to have different categories. In this case, we will create the first category as "all", representing all stocks<br><br>

In [6]:
df2["all"] = "all"

## **Define different categories for the stock returns**

<font size="3">Since we will be showing the stock returns in different colors, it is a good idea to make categories for those returns. The reason, is that you might have very high daily returns for some random stocks, and then it will have a very strong color compared to the rest of stocks. For instance, if you have a +30% return in a single stock, then all the remaining that have returns between 1% and 2% won't show up very clearly in your chart. That's why we do what is shown in the scrip below. If you want to understand the logic behind that please look <a href="https://quant-trading.co/mean-and-median-return-calculations-using-python/" target="_blank">here</a>  <br><br>

In [13]:
conditions = [ (df2['daily_return'] >= -50)&(df2['daily_return'] < -20) , 
               (df2['daily_return'] >= -20)&(df2['daily_return'] < -10) , 
               (df2['daily_return'] >= -10)&(df2['daily_return'] < -5) , 
               (df2['daily_return'] >= -5)&(df2['daily_return'] < -0.01) , 
               (df2['daily_return'] >= 0.01)&(df2['daily_return'] < 1) , 
               (df2['daily_return'] >= 1)&(df2['daily_return'] < 5) , 
               (df2['daily_return'] >= 5)&(df2['daily_return'] < 10) , 
              (df2['daily_return'] >= 10)&(df2['daily_return'] < 20),
              (df2['daily_return'] >= 20)&(df2['daily_return'] < 100)
               ]     

choices = [1,2,3,4,5,6,7,8,9]

df2['daily_return2'] = np.select(conditions, choices, default=5)


df2

,ticker,mkt_cap,daily_return,sector,all,daily_return2,daily_return_str,ticker_plus_return
0,AAPL,3661198000128,-1.138776,Technology,all,4,-1.14,AAPL -1.14%
1,MSFT,3140270161920,-1.280823,Technology,all,4,-1.28,MSFT -1.28%
2,NVDA,3432028569600,-6.216958,Technology,all,3,-6.22,NVDA -6.22%
3,AMZN,2335486771200,-2.416414,Consumer Cyclical,all,4,-2.42,AMZN -2.42%
4,META,1559857201152,-1.953348,Communication Services,all,4,-1.95,META -1.95%
...,...,...,...,...,...,...,...,...
500,BIO,9537294336,0.151396,Healthcare,all,5,0.15,BIO 0.15%
501,RL,15060079616,-0.1811,Consumer Cyclical,all,4,-0.18,RL -0.18%
502,PARA,7739991040,-0.365965,Communication Services,all,4,-0.37,PARA -0.37%
503,FOX,21915037696,-0.552369,Communication Services,all,4,-0.55,FOX -0.55%


## **Make the treemap chart**

<font size="3">You can make a treemap chart using the library plotly express and the function treemap. In this case, the path parameter will indicate the different categories to make the grouping. In our case, they will be "all", "sector" and "ticker". The size of the boxes will be determined by the values parameters, which in this case is "mkt_cap". The colors will be defined by the returns, which is this case are "daily_return2" <br><br>

In [21]:
fig = px.treemap(df2,
                 path=['all','sector','ticker'], 
                 values='mkt_cap',
                 color='daily_return2',
                 hover_data=['daily_return'],
                 color_continuous_scale=["red", "red","white","green" ,"green"],  
                 color_continuous_midpoint=np.average(df2['daily_return2'], weights=df2['mkt_cap'])                 
                )


fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

## **Improve the treemap chart**

<font size="3">we can make a little improvement of the chart, visualizing the returns inside the box and hiding the bar in the right hand side of the chart. You can do this as following:" <br><br>

In [22]:
df2['daily_return_str'] = df2["daily_return"].apply(lambda x: round(x, 2)).astype('str')
df2['ticker_plus_return'] = df2['ticker'] + " " + df2['daily_return_str'] + "%"
df2

,ticker,mkt_cap,daily_return,sector,all,daily_return2,daily_return_str,ticker_plus_return
0,AAPL,3661198000128,-1.138776,Technology,all,4,-1.14,AAPL -1.14%
1,MSFT,3140270161920,-1.280823,Technology,all,4,-1.28,MSFT -1.28%
2,NVDA,3432028569600,-6.216958,Technology,all,3,-6.22,NVDA -6.22%
3,AMZN,2335486771200,-2.416414,Consumer Cyclical,all,4,-2.42,AMZN -2.42%
4,META,1559857201152,-1.953348,Communication Services,all,4,-1.95,META -1.95%
...,...,...,...,...,...,...,...,...
500,BIO,9537294336,0.151396,Healthcare,all,5,0.15,BIO 0.15%
501,RL,15060079616,-0.1811,Consumer Cyclical,all,4,-0.18,RL -0.18%
502,PARA,7739991040,-0.365965,Communication Services,all,4,-0.37,PARA -0.37%
503,FOX,21915037696,-0.552369,Communication Services,all,4,-0.55,FOX -0.55%


In [31]:
fig = px.treemap(df2,
                 path=['all','sector','ticker_plus_return'], 
                 values='mkt_cap',
                 color='daily_return2',
                 hover_data=['daily_return'],
                 color_continuous_scale=["darkred","red","lightgreen","green","darkgreen"],  
                 color_continuous_midpoint=5
                )


fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update(layout_coloraxis_showscale=False)
fig.show()

If this content is helpful and you want to make a donation please click on the button

[![paypal](https://www.paypalobjects.com/en_US/i/btn/btn_donateCC_LG.gif)](https://www.paypal.com/cgi-bin/webscr?cmd=_s-xclick&hosted_button_id=29CVY97MEQ9BY)